# UserProxyAgent using FastAPI

This notebook is a walk-through a modified version of:

- https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_fastapi
To run, open a terminal in Jupyter and execute:


## Running the app

```bash
pip install -r requirements.txt
cd 03_userproxyagent
python app_team.py
```

Open a new browser tab to http://localhost:8002

You should see output in the terminal similar to:

```bash
jovyan@9280d871b652:~/03_userproxyagent$ python app_team.py 
INFO:     Started server process [4400]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)
```

## Terminating the app

Head back to the Jupyter terminal, and press `CTRL-C`.  

You may need to do this a couple of times.  Ignore any error messages.

## Documentation of the app code

This section provides an overview of the application code base in the `work/03_userproxyagent` directory, focusing on the files `app_team.py` and `app_team.html`. Special emphasis is placed on the role of the `UserProxyAgent` within the application.

---

### Overview

The application is a FastAPI-based chat server that hosts a team of agents interacting in a round-robin fashion. The team consists of:

- **AssistantAgent**: A helpful assistant agent.
- **Yoda**: A variant of the assistant agent that repeats messages in the tone of Yoda.
- **UserProxyAgent**: A proxy agent that represents the user, facilitating asynchronous user input via WebSocket.

The front-end interface (`app_team.html`) connects to the backend WebSocket endpoint to send user messages and receive agent responses, enabling real-time chat interaction.

---

### app_team.py

**FastAPI Setup**

- The FastAPI app serves the chat interface HTML (`app_team.html`) at the root endpoint (`/`).
- CORS middleware is configured to allow all origins, methods, and headers.
- Static files are served from the current directory under the `/static` path.

**Team Setup (`get_team` function)**

- Loads model configuration from `model_config.yaml`.
- Instantiates a `ChatCompletionClient` with the loaded config and environment API key.
- Creates three agents:
  - `AssistantAgent` named `"assistant"` with a system message defining its role.
  - `AssistantAgent` named `"yoda"` with a system message to mimic Yoda's tone.
  - `UserProxyAgent` named `"user"` which uses a provided asynchronous input function to receive user input.
- These agents are combined into a `RoundRobinGroupChat` team.
- The team state is loaded from `team_state.json` if it exists.

**UserProxyAgent Role**

- The `UserProxyAgent` acts as a proxy for the human user within the team.
- It is instantiated with an asynchronous input function (`input_func`) that is called whenever the agent requires user input.
- This input function is implemented to asynchronously wait for messages from the WebSocket message queue.
- This design allows the team to request user input asynchronously during the chat flow, enabling interactive conversations.

**WebSocket Chat Endpoint (`/ws/chat`)**

- Accepts WebSocket connections from clients.
- Maintains an asynchronous queue (`message_queue`) to hold incoming user messages.
- Defines an internal `_user_input` async function that the `UserProxyAgent` uses to get user input by awaiting messages from the queue.
- Listens for incoming WebSocket messages, validates them as `TextMessage` objects, and enqueues them.
- Runs the team chat logic in a streaming fashion, sending agent messages back to the client over the WebSocket.
- Saves team state and chat history to JSON files after each interaction.
- Handles disconnections and errors gracefully, sending error messages to the client as needed.

---

### app_team.html

**Front-End Chat Interface**

- A simple HTML page with a chat container displaying messages and an input box with a send button.
- Connects to the backend WebSocket endpoint at `ws://localhost:8002/ws/chat`.
- Sends user messages to the WebSocket when the send button is clicked or Enter is pressed.
- Displays incoming messages from the WebSocket, differentiating between user, assistant, system, and error messages.
- Handles special message types like `UserInputRequestedEvent` to re-enable the input box when the backend requests user input.
- Loads chat history from the backend on page load to display previous messages.

---

### Summary: UserProxyAgent's Role

The `UserProxyAgent` serves as the bridge between the human user and the agent team. It enables the team to request and receive user input asynchronously during the conversation. This is achieved by:

- Passing an asynchronous input function to the `UserProxyAgent` that awaits messages from the WebSocket message queue.
- The WebSocket endpoint receives user messages from the front-end and enqueues them for consumption by the `UserProxyAgent`.
- This design allows the team to operate in a round-robin manner, with the `UserProxyAgent` seamlessly integrating human input into the agent conversation flow.

This architecture enables interactive, real-time chat experiences where AI agents and the user collaborate in a conversational team.
